In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH05-Model")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Model


# Serialization 

1. 개요 
LLM 모델 객체를 저장 가능한 형태로 변환하는 과정 
    
2. 목적
    - 모델 재사용 (재훈련 없음) 
    - 모델 배포 및 공유 
    - 계산 리소스 절약 

In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

In [4]:
# 직렬화 가능여부 확인 
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [5]:
# 모델의 직렬화 가능여부 확인 
model = ChatOpenAI()
print(f"ChatOpenAI: {model.is_lc_serializable()}")

ChatOpenAI: True


In [6]:
# 체인의 직렬화 가능여부 확인 
prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")
chain = prompt | model 
print(f"ChatOpenAI: {chain.is_lc_serializable()}")

ChatOpenAI: True


# 체인 직열화 

- 객체의 속성 및 데이터를 키-값 쌍으로 저장하여 딕셔너리 형태로 변환  
- 방식
    - `dumps`: 객체를 JSON 문자열로 직렬화 (s: str)
    - `dumpd`: 객체를 딕셔너리 형태로 직렬화 (d: dict)

In [7]:
from langchain_core.load import dumps, dumpd

In [8]:
# 딕셔너리 형태(d) 직렬화
chain_dumpd = dumpd(chain)
chain_dumpd

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'temperature': 0.7,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'max_retries': 2,
    'n': 1},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

In [9]:
type(chain_dumpd)

dict

In [10]:
# JSON 형태(s) 직렬화
chain_dumps = dumps(chain)
chain_dumps

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "gpt-3.5-turbo", "temperature": 0.7, "openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}, "max_retries": 2, "n": 1}, "name": "ChatOpenAI"}}, "name": "RunnableSequence"}'

In [11]:
type(chain_dumps)

str

# Pickle 

- Python 객체를 바이너리 형태로 직렬화 
- 주요 용도 
    - 객체 캐싱 
    - AI 모델 저장 
    - 프로그램 저장/복원 
- 방식 
    - `pickle.dump()`: 객체를 파일에 저장
    - `pickle.load()`: 파일에서 객체 로드

In [16]:
import pickle
from langchain_core.load import load

In [13]:
with open('chain_fruit.pkl', 'wb') as f:
    pickle.dump(chain_dumpd, f)

In [14]:
# 저장된 pickle 파일 불러오기 
with open('chain_fruit.pkl', 'rb') as f:
    chain_loaded = pickle.load(f)

In [15]:
chain_loaded

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'temperature': 0.7,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'max_retries': 2,
    'n': 1},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

In [18]:
# 파일 로딩 
chain_loaded_file = load(chain_loaded)

# 체인 실행
answer = chain_loaded_file.invoke({"fruit": "사과"})
print(answer.content)

사과의 색상은 주로 빨간색이지만, 녹색, 황색 또는 주황색일 수도 있습니다.


JSON 형태로 저장, 로딩, 실행

In [19]:
import json

In [20]:
with open("chain_fruit.json", "w") as fp:
    json.dump(chain_dumpd, fp)

In [21]:
with open("chain_fruit.json", "r") as fp:
    chain_loaded_json = json.load(fp)
    chain_loaded= load(chain_loaded_json)

In [22]:
answer = chain_loaded.invoke({"fruit": "한라봉"})
print(answer.content)

한라봉은 주황색을 띠고 있습니다.


-----
** End of Documents **